In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("dataset/combined_GH_HF_manual.csv")

# Function to clean text columns
def clean_text(text):
    # Remove non-ASCII characters (corrupted/malformed characters)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    return text.strip()

# Apply cleaning function to relevant columns
df['message'] = df['message'].apply(clean_text)
df = df[df['message'].notna() & (df['message'] != '') & (df['message'].str.split().str.len() > 1)]
df["label"] = df["label"].str.lower()

number_of_labels = df["label"].value_counts()
number_of_labels

label
external documentation       501
model structure              195
project metadata             141
sharing                      127
preprocessing                 61
training infrastructure       55
validation infrastructure     52
input data                    35
internal documentation        35
pipeline performance          33
parameter tuning              31
add dependency                19
output data                   18
update dependency             17
remove dependency             15
Name: count, dtype: int64

In [4]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(df["label"]), 
    y=df["label"]
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Check class weights
print("Class Weights:", class_weights)

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Custom model with class weights
class DistilBERTWithWeightedLoss(nn.Module):
    def __init__(self, model_name, num_labels, class_weights, dropout=0.2):
        super().__init__()
        
        # Create a custom configuration with modified dropout
        config = DistilBertConfig.from_pretrained(
            model_name, 
            num_labels=num_labels, 
            hidden_dropout_prob=dropout,  # Dropout for feedforward layers
            attention_probs_dropout_prob=dropout  # Dropout for attention layers
        )

        # Load the model with the custom config
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)  # Apply class weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Set device (MPS for Apple Silicon, else fallback to CPU)
device = torch.device("mps" if torch.has_mps else "cpu")
print(f"Using device: {device}")

# Initialize the model
num_labels = len(label_encoder.classes_)  # Count unique labels
model = DistilBERTWithWeightedLoss(model_name, num_labels, class_weights, dropout=0.3)
model.to(device)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Keep only the best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # Lower loss is better
    learning_rate=1e-5,  # Lower learning rate for stability
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,  # Keep the number of epochs as 6
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_steps=1000,  # Log less frequently to reduce clutter
    report_to="none",  # Prevents reporting to WandB, TensorBoard, etc.
    fp16=False,  # Disable mixed precision to avoid MPS issue
    warmup_ratio=0.06,  # Gradually warm up the learning rate
    gradient_accumulation_steps=4,  # Helps stabilize training
)

# Compute metrics (Accuracy, Precision, Recall, F1)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=1
    )
    acc = accuracy_score(labels, predictions)

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Start Training
trainer.train()

Class Weights: tensor([4.6842, 0.1776, 2.5429, 2.5429, 0.4564, 4.9444, 2.8710, 2.6970, 1.4590,
        0.6312, 5.9333, 0.7008, 1.6182, 5.2353, 1.7115])


/Users/adekunleajibode/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Map:   0%|          | 0/1068 [00:00<?, ? examples/s]

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

/var/folders/4p/p4dff74s5rb42f27sxjfg97r0000gn/T/ipykernel_96037/3001673688.py:61: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: mps


/Users/adekunleajibode/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.678300,2.695112,0.247191,0.823893,0.247191,0.408120
2,2.638000,2.668528,0.415730,0.593485,0.415730,0.569209
3,2.601100,2.638953,0.456929,0.593988,0.456929,0.473263
4,2.562500,2.614130,0.460674,0.598915,0.460674,0.482343


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
import random
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.nn import CrossEntropyLoss
import nltk

df = pd.read_csv("dataset/combined_GH_HF_manual.csv")

# Function to clean text columns
def clean_text(text):
    # Remove non-ASCII characters (corrupted/malformed characters)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    return text.strip()

# Apply cleaning function to relevant columns
df['message'] = df['message'].apply(clean_text)
df = df[df['message'].notna() & (df['message'] != '') & (df['message'].str.split().str.len() > 1)]
df["label"] = df["label"].str.lower()

number_of_labels = df["label"].value_counts()
number_of_labels

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(df["label"]), 
    y=df["label"]
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Check class weights
print("Class Weights:", class_weights)


# First, split into training (80%) and testing (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Then, split the training set into training (80% of 80% = 64%) and validation (20% of 80% = 16%)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Ensure labels are of type integer
train_dataset = train_dataset.map(lambda e: {'label': int(e['label'])})
val_dataset = val_dataset.map(lambda e: {'label': int(e['label'])})
test_dataset = test_dataset.map(lambda e: {'label': int(e['label'])})

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

class DistilBERTWithWeightedLoss(nn.Module):
    def __init__(self, model_name, num_labels, class_weights, dropout=0.4):
        super().__init__()
        
        # Create a custom configuration with modified dropout
        config = DistilBertConfig.from_pretrained(
            model_name, 
            num_labels=num_labels, 
            hidden_dropout_prob=dropout,  # Dropout for feedforward layers
            attention_probs_dropout_prob=dropout  # Dropout for attention layers
        )

        # Load the model with the custom config
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)  # Apply class weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}
    
device = torch.device("mps" if torch.has_mps else "cpu")
print(f"Using device: {device}")

# Initialize model
num_labels = len(label_encoder.classes_)  # Count unique labels
model = DistilBERTWithWeightedLoss(model_name, num_labels, class_weights, dropout=0.2)
model.to(device)



# Initialize model
num_labels = len(label_encoder.classes_)  # Count unique labels
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBERTWithWeightedLoss(model_name, num_labels, class_weights, dropout=0.4)
model.to(device)  # Move model to GPU if available

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Keep only the best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # Lower loss is better
    learning_rate= 2e-5,  # Lower learning rate to improve stability
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,  # Reduce epochs to prevent overfitting
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_steps=1000,  # Log less frequently to reduce clutter
    report_to="none",  # Prevents reporting to WandB, TensorBoard, etc.
    fp16=False,  # Disable mixed precision to avoid MPS issue
    warmup_ratio=0.1,  # Warm-up for the first 10% of training
    lr_scheduler_type="linear",
    gradient_accumulation_steps=2,  # Helps stabilize training
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=1
    )
    acc = accuracy_score(labels, predictions)

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")